In [56]:
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop


In [57]:
text = open('1661-0.txt').read().lower()

In [58]:
len(text)

806343

In [59]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(text)

In [60]:
unique_tokens = np.unique(tokens)

In [61]:
unique_tokens_index = dict((c, i) for i, c in enumerate(unique_tokens))


In [62]:
n_words = 5
input_words = []
next_words = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_words.append(tokens[i+n_words])

In [63]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [64]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_tokens_index[word]] = 1
    Y[i, unique_tokens_index[next_words[i]]] = 1

In [65]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens))))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [66]:
model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history

Epoch 1/2
1110/1110 [==============================] - 89s 80ms/step - loss: 6.1042 - accuracy: 0.1099 - val_loss: 7.4812 - val_accuracy: 0.1034
Epoch 2/2
1110/1110 [==============================] - 83s 75ms/step - loss: 6.0400 - accuracy: 0.1505 - val_loss: 8.0699 - val_accuracy: 0.1043


In [67]:
model.save("mymodel.h5")
model = load_model("mymodel.h5")


In [68]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_tokens_index[word]] = 1

    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [70]:
possible = predict_next_word("Can you feel the love", 5)

1/1 [==============================] - 0s 179ms/step


In [71]:
print([unique_tokens[idx] for idx in possible])

['that', 'and', 'for', 'to', 'of']


In [72]:
def generate_text(input_text,text_length, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(text_length):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            print("here")
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [74]:
generate_text("Can you feel the love", 20, 1)

1/1 [==============================] - 0s 12ms/step


'Can you feel the love of the and the man who was in the room and i was a very much more than the matter'